# Example Property Pro Forma
This notebook contains a simple office valuation model. This rudamentary model highlights a few key benefits of a programmatic model over a typical Excel workbook, specifically:
* Database connection--Property and in-place lease data are queried from a database which mirrors the abilty to draw data directly from a property management or other data management system
* Web API consumption--Any resource available over the web can be easily accessed, in this case a simple model for market lease assumptions
* Scalability--Pro formas for multiple buildings can be created in just a couple lines of code

This notebook doesn't provide a complete modeling framework, but it offers a glimpse at a different way to underwrite.

In [2]:
import pandas as pd
from leases import GrossLease
from properties import OfficeProperty
import requests
from seriesgroup import SeriesGroup
import sqlite3
from utils import memoized_series

Get property and existing lease data from the database. The building's name is **Office Tower A**.

In [3]:
building_name = 'Office Tower A'

with sqlite3.connect('properties_database.db') as db:
    db.row_factory = sqlite3.Row
    cur = db.cursor()
    property_data = dict(cur.execute('select * from properties where name like (?)', (building_name,)).fetchone())

    expenses = dict(cur.execute('select cam_psf, insurance_psf, utilities_psf, ret_psf, \
                                management_pct from expenses where property_id = (?)', (property_data['id'],))
                               .fetchone())
    occupied_suites = [dict(suite) for suite in cur.execute('select sf, tenant_name as name, start, term, rent_psf, \
                                                            free_periods, escalation_pct, escalation_freq from spaces \
                                                            left join leases on spaces.id = leases.suite_id \
                                                            where property_id = (?) and tenant_name is not null', 
                                                            (property_data['id'],))]
    vacant_suites = [dict(suite) for suite in cur.execute('select suite as name, sf, floor from spaces \
                                                          left join leases on spaces.id = leases.suite_id \
                                                          where spaces.property_id = (?) and tenant_name is null', 
                                                          (property_data['id'],))]

In [4]:
in_place_leases = [GrossLease(**lease) for lease in occupied_suites]

The following cell highlights how you can integrate third-party models directly into the valuation model. There are cool, novel approaches that are enabled by the fact that you can work with any type of data. For example, passing floor plate images to an automated test fit model could automatically identify and suites/floors that require rent discounts due to awkward or inefficient layouts. 

For this example, the [market leasing assumptions API](https://leasingassumptions.herokuapp.com/docs) is a highly sophisticated model that returns a market rate per square foot of 40 + 0.25 per floor. Note that it may be taken down at any time, and it runs on a Heroku free tier meaning that it may take a while to wake up if it hasn't been accessed recently.

In [5]:
leasing_assumptions = [requests.get('https://leasingassumptions.herokuapp.com/marketleaserates', 
                                    params={'address': property_data['address'], 'floor': suite['floor']}).json() 
                       for suite in vacant_suites]
print(f'Example of market rate lease data: \n{leasing_assumptions[0]}')
speculative_leases = [GrossLease(name=suite['name'], sf=suite['sf'], **assumptions) 
                      for suite, assumptions in zip(vacant_suites, leasing_assumptions)]

Example of market rate lease data: 
{'start': 1, 'free_periods': 0, 'term': 5, 'rent_psf': 40.5, 'escalation_pct': 0.03, 'escalation_freq': 1}


Build the pro forma using the property expense data, existing leases, and market rate assumptions.

In [6]:
office_tower_a = OfficeProperty(name=property_data['name'], 
                                sf=sum([l['sf'] for l in [*occupied_suites, *vacant_suites]]),
                                leases=SeriesGroup.with_series('leases', [*in_place_leases, *speculative_leases]),
                                **expenses)

In [7]:
pd.DataFrame([office_tower_a(year) for year in range(6)]).T.style.format('{:,.0f}')

,0,1,2,3,4,5
leases.Suite200.potential_rent,0,"1,042,875","1,074,161","1,106,386","1,139,578","1,173,765"
leases.Suite200.free_rent,0,0,0,0,0,0
leases.Suite200.effective_rent,0,"1,042,875","1,074,161","1,106,386","1,139,578","1,173,765"
leases.Suite400.potential_rent,0,"844,600","869,938","896,036","922,917","950,605"
leases.Suite400.free_rent,0,0,0,0,0,0
leases.Suite400.effective_rent,0,"844,600","869,938","896,036","922,917","950,605"
leases.Tenant A.potential_rent,"1,050,000","1,076,250","1,103,156","1,130,735","1,159,004","1,187,979"
leases.Tenant A.free_rent,0,0,0,0,0,0
leases.Tenant A.effective_rent,"1,050,000","1,076,250","1,103,156","1,130,735","1,159,004","1,187,979"
leases.Tenant b.potential_rent,"940,000","940,000","940,000","940,000","940,000","987,000"


The exact same code from the cells above was dumped into a function called `build_office` that takes a single property name parameter. The cell below shows how code can be reused to efficiently build projections.

In [9]:
from properties import build_office

office_tower_b = build_office('Office Tower B')
pd.DataFrame([office_tower_b(year) for year in range(6)]).T.style.format('{:,.0f}')

,0,1,2,3,4,5
leases.Suite205.potential_rent,0,"1,042,875","1,074,161","1,106,386","1,139,578","1,173,765"
leases.Suite205.free_rent,0,0,0,0,0,0
leases.Suite205.effective_rent,0,"1,042,875","1,074,161","1,106,386","1,139,578","1,173,765"
leases.TenantC.potential_rent,"2,850,000","2,850,000","2,850,000","2,850,000","2,850,000","2,992,500"
leases.TenantC.free_rent,0,0,0,0,0,0
leases.TenantC.effective_rent,"2,850,000","2,850,000","2,850,000","2,850,000","2,850,000","2,992,500"
leases.TenantD.potential_rent,"1,125,000","1,136,250","1,147,612","1,159,089","1,170,680","1,182,386"
leases.TenantD.free_rent,0,0,0,0,0,0
leases.TenantD.effective_rent,"1,125,000","1,136,250","1,147,612","1,159,089","1,170,680","1,182,386"
egi,"3,975,000","5,029,125","5,071,774","5,115,475","5,160,257","5,348,651"
